In [ ]:
#Load in all libraries

from azure.identity import AzureCliCredential
from azure.storage.filedatalake import DataLakeFileClient
from azure.storage.filedatalake import DataLakeServiceClient
import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from datetime import datetime, timedelta
import plotly.express as px
import re
from pulp import LpProblem, LpMinimize, LpMaximize, LpVariable, lpSum, LpStatus, value, LpBinary

In [ ]:
#Azure functions

def get_credential():
    # Haal het token op wat met az login is aangemaakt
    return AzureCliCredential()


def read_file(credential, storage_account, container,
                  filepath):
    account_url = "https://{}.dfs.core.windows.net".format(storage_account)

    file_client = DataLakeFileClient(account_url=account_url,
                                     file_system_name=container,
                                     file_path=filepath,
                                     credential=credential)

    downloaded_bytes = io.BytesIO(file_client.download_file().readall())
    return downloaded_bytes


def write_file(buffer, credential, storage_account, container,
                filepath):
    account_url = "https://{}.dfs.core.windows.net".format(storage_account)

    file_client = DataLakeFileClient(account_url=account_url,
                                     file_system_name=container,
                                     file_path=filepath,
                                     credential=credential)

    file_client.upload_data(buffer.getvalue(), overwrite=True)

In [ ]:
# Request token
credential = get_credential()

# Definieer storage-account en containername
storage_account_name = "prda007itweup01dapsts01"
container_name = "ait-analytics30-s"
account_url = "https://{}.dfs.core.windows.net".format(storage_account_name)
reports_folder = "426_rapportage/"
schedule_folder = "werkvoorraad_planning/"
schedule2_folder = "werkverdeling_acceptatie/"

In [ ]:
# Read file
import os
import pyarrow.parquet as pq

def get_df(folder = "426_rapportage/", file_name = "426_werkvoorraad_MO_20230131_AMB.xlsx"):
    file_path = folder + '/' + file_name

    file_bytes = read_file(credential=credential,
                  storage_account=storage_account_name,
                  container=container_name,
                  filepath=file_path)
    
    return(file_bytes)

In [ ]:
#function to remove prefix
def remove_prefix(text, prefix):
    if text.startswith(prefix):
        return text[len(prefix):]
    return text

#Function to retrieve all file names from a folder
def get_all_file_names(account_url, credential, container_name, folder):
    # Get a reference to the file system and directory
    service_client = DataLakeServiceClient(account_url=account_url, credential=credential)
    file_system_client = service_client.get_file_system_client(container_name)

    file_list = []
    # List files in the directory
    files = file_system_client.get_paths()
    for file in files:
        if ((file.name.startswith(folder)) & (".xlsx" in file.name)):
            file_list.append(file.name)
    return file_list

In [ ]:
# Function to extract filename with the specific date
def extract_filename_with_date(filenames, date):
    for filename in filenames:
        if date in filename:
            return filename
    return None  # Return None if no matching filename is found

# Function to convert YYYYMMDD to DD-MM-YYYY
def convert_date_format(yyyymmdd):
    return f"{yyyymmdd[6:8]}-{yyyymmdd[4:6]}-{yyyymmdd[0:4]}"

# Function to extract filename with the specific date, considering the new format
def extract_filename_with_converted_date(filenames, date):
    for filename in filenames:
        # Use regular expressions to find the date in YYYYMMDD format within the filename
        match = re.search(r'\d{8}', filename)
        if match:
            date_in_file = match.group(0)  # Extract the date string
            converted_date = convert_date_format(date_in_file)  # Convert to DD-MM-YYYY
            if converted_date == date:
                return filename

## Load data into environment

In [ ]:
#Function to create an aggregated dataframe of the employee availability.
def transform_schedule(df, skill_lookup):
    weekly_schedule_df = df.copy()

    # Identifying team names and their respective employee columns
    team_columns = [col for col in weekly_schedule_df.columns if 'Team' in col]

    # Mapping each team to its members
    team_member_map = {}
    for i, team_col in enumerate(team_columns):
        team_idx = weekly_schedule_df.columns.get_loc(team_col)
        if i < len(team_columns) - 1:
            next_team_idx = weekly_schedule_df.columns.get_loc(team_columns[i + 1])
            members = weekly_schedule_df.columns[team_idx + 1:next_team_idx]
        else:
            members = weekly_schedule_df.columns[team_idx + 1:]

        team_member_map[weekly_schedule_df[team_col].iloc[0]] = members

    # Transforming the dataframe
    new_data = []
    for index, row in weekly_schedule_df.iterrows():
        for team_name, team_members in team_member_map.items():
            total_hours = row[team_members].apply(pd.to_numeric, errors='coerce').sum()
            all_skills_hours = 0
            not_all_skills_hours = 0

            for member in team_members:
                first_name = member
                hours = pd.to_numeric(row[first_name], errors='coerce')
                if pd.isna(hours):
                    hours = 0  # Set NaN values to 0
                skill = skill_lookup.get(first_name, 'Acceptant A')  # Default to 'Acceptant A' if not found
                
                if skill == 'Acceptant B':
                    not_all_skills_hours += hours
                else:
                    all_skills_hours += hours

            new_data.append({
                'Datum': row['Datum'],
                'Dag': row['Dag'],
                'Team': team_name,
                'TotalHours': total_hours,
                'AllSkillsHours': all_skills_hours,
                'NotAllSkillsHours': not_all_skills_hours
            })

    # Create the new dataframe
    transformed_df = pd.DataFrame(new_data)

    # Display the first few rows of the new dataframe
    return transformed_df

In [ ]:
#Load in skill matrix and make lookup file
skill_matrix = pd.read_excel("Skillmatrix.xlsx", skiprows=2)

def extract_first_name(full_name):
    return full_name.split()[0]

# Step 1: Extract first names and create a mapping from first name to full name and skill
skill_matrix['FirstName'] = skill_matrix['Naam'].apply(extract_first_name)

# Step 2: Create a dictionary for skill lookup
skill_lookup = skill_matrix.set_index('FirstName')['Medewerkerprofiel'].to_dict()

In [ ]:
#Get latest working schedule
schedule_file_names = get_all_file_names(account_url, credential, container_name, folder = schedule_folder)
schedule_file_names2 = get_all_file_names(account_url, credential, container_name, folder = schedule2_folder)

weekly_schedules = []
for i in schedule_file_names:
    split_name = i.rsplit('/',1)
    excel_file = get_df(folder=split_name[0] + '/', file_name=split_name[1])
    # Check if the 'ROOSTER' sheet exists in the Excel file
    if 'ROOSTER' in pd.ExcelFile(excel_file).sheet_names:
        temp = pd.read_excel(excel_file, sheet_name='ROOSTER')
        temp['Datum'] = pd.to_datetime(temp['Datum'], errors='coerce')
        temp = temp.dropna(subset=['Datum'])
        temp = temp.dropna(axis=1, how='all')
        transformed_temp = transform_schedule(temp, skill_lookup)
        weekly_schedules.append(transformed_temp)
        # weekly_schedules.append(temp)
    else:
        print(f"Skipping file {i} because 'ROOSTER' sheet doesn't exist.")

for i in schedule_file_names2:
    split_name = i.rsplit('/',1)
    excel_file = get_df(folder=split_name[0] + '/', file_name=split_name[1])
    # Check if the 'ROOSTER' sheet exists in the Excel file
    if 'ROOSTER' in pd.ExcelFile(excel_file).sheet_names:
        temp = pd.read_excel(excel_file, sheet_name='ROOSTER')
        temp['Datum'] = pd.to_datetime(temp['Datum'], errors='coerce')
        temp = temp.dropna(subset=['Datum'])
        temp = temp.dropna(axis=1, how='all')
        transformed_temp = transform_schedule(temp, skill_lookup)
        weekly_schedules.append(transformed_temp)
    else:
        print(f"Skipping file {i} because 'ROOSTER' sheet doesn't exist.")        

schedule = pd.concat(weekly_schedules)

In [ ]:
#Get task groups
task_groups = pd.read_excel('Procesgroep taken acceptatie.xlsx', sheet_name ='Lijstvorm')
task_groups = task_groups[['Procesgroep', 'Proces']]

In [ ]:
#Get backlog of all tasks
report_file_names = get_all_file_names(account_url, credential, container_name, folder = reports_folder)

reports = []
for i in report_file_names:
    split_name = i.rsplit('/',1)
    excel_file = get_df(folder=split_name[0] + '/', file_name=split_name[1])
    date = split_name[1].rsplit('MO_', 1)[1].rsplit('_A')[0]
    temp = pd.read_excel(excel_file)
    temp['datum'] = pd.to_datetime(split_name[1].rsplit('MO_', 1)[1].rsplit('_A')[0], format='%Y%m%d').strftime('%Y-%m-%d')
    reports.append(temp)

backlog = pd.concat(reports)

## Prepare data

In [ ]:
#Clean working backlog
backlog = backlog[backlog['contractnummer'].notna()]
backlog['contractnummer'] =  backlog['contractnummer'].astype(np.int64)
backlog['datum'] = pd.to_datetime(backlog['datum'])
backlog = backlog.merge(task_groups, left_on='taaknaam', right_on='Proces', how='left')
filtered_backlog = backlog.copy()
# filtered_backlog = backlog[['datum', 'contractnummer', 'Procesgroep_y', 'taaknaam', 'taakomschrijving_aangepast', 'teamcode', 'Uitvoeren voor']]
filtered_backlog.rename(columns={'Procesgroep_y': 'Procesgroep'}, inplace=True)
filtered_backlog = filtered_backlog[filtered_backlog['Procesgroep'] != 'Quion']
filtered_backlog['week_nummer'] = filtered_backlog['datum'].dt.isocalendar().week

# Define the list of team codes you want to keep
team_codes_to_keep = ['Zuid', 'Midden', 'Noord', 'IMD']

# Use the isin method to filter the DataFrame
filtered_backlog = filtered_backlog[filtered_backlog['teamcode'].isin(team_codes_to_keep)]
filtered_backlog.loc[filtered_backlog['productlijn'] == 'Attens Hypotheek', 'teamcode'] = 'Attens'
filtered_backlog = filtered_backlog[filtered_backlog["teamcode"] != "IMD"]
# Group by 'procesgroep' and 'contractnummer', then count unique 'taaknaam' for each group.
unique_tasks_per_group_and_contract = filtered_backlog.groupby(['Procesgroep', 'contractnummer'])['Proces'].nunique().reset_index(name='unique_tasks')

# Now, group by 'procesgroep' again to calculate the average number of unique tasks per procesgroep.
average_unique_tasks_per_procesgroep = unique_tasks_per_group_and_contract.groupby('Procesgroep')['unique_tasks'].mean()

# Define the column names
column_names = ['Procesgroep', 'Normtijd (in minuten)']

# Manually enter the rows with data
rows = [
    ["Aanvragen", 16.8],
    ["Rebound", 21.0],
    ["1e fiat", 55.8],
    ["2e fiat", 27.0],
    ['Afronding dossier', 10.0/2],
    ['Schoningstaken', 10.0]
]

# Create the dataframe
task_times = pd.DataFrame(rows, columns=column_names)

filtered_backlog = filtered_backlog.merge(task_times, left_on="Procesgroep", right_on="Procesgroep", how="left")
filtered_backlog['Normtijd (in minuten)'] = (filtered_backlog['Normtijd (in minuten)'] / 60).round(2)
filtered_backlog.rename(columns={'Normtijd (in minuten)': 'Normtijd (in hours)'}, inplace=True)
filtered_backlog['datum'] = filtered_backlog['datum'] + timedelta(days=1)

In [ ]:
#Clean schedule
schedule = schedule.drop_duplicates()
schedule = schedule.fillna(0)

# List of columns to exclude from replacement
exclude_columns = ['Datum', 'Dag', 'Team']

# List of columns representing employee hours
employee_columns = [col for col in schedule.columns if col not in exclude_columns]

# Apply the replacement only to the employee columns
schedule[employee_columns] = schedule[employee_columns].apply(pd.to_numeric, errors='coerce').fillna(0)

team_codes_to_keep = ['Zuid', 'Midden', 'Noord', 'Attens']
schedule = schedule[schedule['Team'].isin(team_codes_to_keep)]

schedule = schedule.drop_duplicates(subset=['Datum', 'Dag', 'Team'], keep='first')

In [ ]:
def solveIP(filtered_backlog, schedule, start_date_str, end_date_str):
    # Define the date range for the analysis
    start_date = datetime.strptime(start_date_str, "%d-%m-%Y")
    end_date = datetime.strptime(end_date_str, "%d-%m-%Y")

    # Filter the dataframes
    filtered_backlog_partly = filtered_backlog[(filtered_backlog['datum'] >= start_date) & (filtered_backlog['Uitvoeren voor'] <= end_date)]
    schedule_partly = schedule[(schedule['Datum'] >= start_date) & (schedule['Datum'] <= end_date)]
    dates = pd.date_range(start=schedule_partly["Datum"].min(), end=schedule_partly["Datum"].max(), freq='D')

    # Initialize the model
    model = LpProblem("Workforce_Scheduling", LpMaximize)

    # Decision Variables
    filtered_backlog_partly['case_number_task'] = filtered_backlog_partly['contractnummer'].astype(str) + '_' + filtered_backlog_partly['taaknaam'].astype(str) + '_' + filtered_backlog_partly['Taakgroep'].astype(str)
    filtered_backlog_partly['case_number_task'] = filtered_backlog_partly['case_number_task'].apply(lambda x: x.replace(" ", "_").replace("/", "_"))

    all_tasks = filtered_backlog_partly['case_number_task'].unique()
    x = LpVariable.dicts("x", [(i, k) for i in all_tasks for k in dates], cat='Binary')

    # Objective Function: Maximize the number of tasks completed
    model += lpSum(x[i, k] for i in all_tasks for k in dates)

    # Constraint: Each task must be assigned exactly once
    for i in all_tasks:
        model += lpSum(x[i, k] for k in dates) == 1

    # Constraint: No deadlines are crossed
    for i in all_tasks:
        deadline = filtered_backlog_partly.loc[filtered_backlog_partly['case_number_task'] == i, 'Uitvoeren voor'].iloc[0]
        model += lpSum(x[i, k] for k in dates if k <= deadline) == 1    

    # Constraint: Total time cost of tasks assigned to a team on a date does not exceed the available hours
    for j in schedule_partly['Team'].unique():
        for k in dates:
            total_hours_available = schedule_partly[(schedule_partly['Datum'] == k) & (schedule_partly['Team'] == j)]['TotalHours'].values[0]
            all_skills_hours_available = schedule_partly[(schedule_partly['Datum'] == k) & (schedule_partly['Team'] == j)]['AllSkillsHours'].values[0]
            not_all_skills_hours_available = schedule_partly[(schedule_partly['Datum'] == k) & (schedule_partly['Team'] == j)]['NotAllSkillsHours'].values[0]

            sum_tasks = lpSum(filtered_backlog_partly.loc[filtered_backlog_partly['case_number_task'] == i, 'Normtijd (in hours)'].iloc[0] * x[i, k] 
                              for i in all_tasks if filtered_backlog_partly.loc[filtered_backlog_partly['case_number_task'] == i, 'teamcode'].iloc[0] == j)

            model += sum_tasks <= total_hours_available

            # Constraint for tasks requiring all skills
            sum_all_skills_tasks = lpSum(filtered_backlog_partly.loc[filtered_backlog_partly['case_number_task'] == i, 'Normtijd (in hours)'].iloc[0] * x[i, k]
                                         for i in all_tasks if filtered_backlog_partly.loc[filtered_backlog_partly['case_number_task'] == i, 'teamcode'].iloc[0] == j and
                                         filtered_backlog_partly.loc[filtered_backlog_partly['case_number_task'] == i, 'Procesgroep'].iloc[0] == '2e fiat')
            model += sum_all_skills_tasks <= all_skills_hours_available

            # Constraint for other tasks that can use either type of skill hours
            sum_other_tasks = lpSum(filtered_backlog_partly.loc[filtered_backlog_partly['case_number_task'] == i, 'Normtijd (in hours)'].iloc[0] * x[i, k]
                                    for i in all_tasks if filtered_backlog_partly.loc[filtered_backlog_partly['case_number_task'] == i, 'teamcode'].iloc[0] == j and
                                    filtered_backlog_partly.loc[filtered_backlog_partly['case_number_task'] == i, 'Procesgroep'].iloc[0] != '2e fiat')
            model += sum_other_tasks <= (all_skills_hours_available + not_all_skills_hours_available)

    model.solve()


    def split_string(input_string):
        # Extract the part within single quotes
        part1 = re.search(r"'([^']*)'", input_string)
        if part1:
            part1 = part1.group(1)

        # Extract the timestamp part and convert it to datetime
        part2 = re.search(r"Timestamp\('([^']+)'\)", input_string)
        if part2:
            timestamp_str = part2.group(1)
            # Parse the datetime from the string
            part2 = datetime.strptime(timestamp_str, '%Y_%m_%d_%H:%M:%S')

        return part1, part2

    # Extracting the assignments
    assignments = []
    for v in model.variables():
        if v.varValue > 0.99:  # Filter to only include assigned tasks
            task, date = split_string(str(v))
            team = filtered_backlog_partly.loc[filtered_backlog_partly['case_number_task'] == task, 'teamcode'].iloc[0]
            hours = filtered_backlog_partly.loc[filtered_backlog_partly['case_number_task'] == task, 'Normtijd (in hours)'].iloc[0]
            assignments.append((task, team, date, hours))

    # Convert to DataFrame
    assignments_df = pd.DataFrame(assignments, columns=['Task', 'Team', 'Date', 'Hours'])
    assignments_df['Scheduled'] = 'Yes'

    # Create a DataFrame of all tasks with their deadlines
    all_tasks_df = filtered_backlog_partly[['case_number_task', 'Uitvoeren voor']]
    all_tasks_df.columns = ['Task', 'Deadline']
    all_tasks_df = all_tasks_df.drop_duplicates()

    # Merge to see which tasks were scheduled and which were not
    result_df = pd.merge(all_tasks_df, assignments_df, on='Task', how='left')
    result_df.fillna({'Scheduled': 'No', 'Date': 'Not Scheduled'}, inplace=True)

    # Ensure the task identifier columns are named consistently
    result_df.rename(columns={'Task': 'case_number_task', 'Team': 'teamcode', 'Hours': 'Normtijd (in hours)'}, inplace=True)

    # Perform a left merge to match tasks and bring the 'Team' and 'Hours' columns
    merge_df = pd.merge(result_df, filtered_backlog_partly[['case_number_task', 'teamcode', 'Normtijd (in hours)']],
                        on='case_number_task', how='left', suffixes=('', '_from_backlog'))

    # Fill NaN values in 'Team' and 'Hours' in result_df with the values from merge_df
    result_df['teamcode'] = result_df['teamcode'].fillna(merge_df['teamcode_from_backlog'])
    result_df['Normtijd (in hours)'] = result_df['Normtijd (in hours)'].fillna(merge_df['Normtijd (in hours)_from_backlog'])

    # Drop the extra columns from merge_df if they exist
    result_df.drop(columns=['teamcode_from_backlog', 'Normtijd (in hours)_from_backlog'], errors='ignore', inplace=True)
    
    # Check for duplicate task assignments
    duplicates = result_df[result_df.duplicated(['case_number_task', 'Date'], keep=False)]
    if not duplicates.empty:
        print("Duplicate task assignments detected:")
        print(duplicates)
    
    return result_df

In [ ]:
# Define your start and end date for the date range
start_date = '2024-02-19'
end_date = '2023-03-01'

# Filter the dates within the specified range and exclude weekends
filtered_dates = filtered_backlog[
    (filtered_backlog['datum'] >= start_date) & 
    (filtered_backlog['datum'] <= end_date) &
    (filtered_backlog['datum'].dt.weekday < 5)  # Monday=0, Sunday=6
]["datum"].unique()

# Create an array of dates in the "yyyy-mm-dd" format
date_strings = filtered_dates.strftime('%d-%m-%Y').tolist()

# Initialize an empty list to hold DataFrames
dfs = []

end_date = '01-03-2024'

# Loop over each date in the string format
for start_date_str in date_strings:
    # Call the solveIP function
    res = solveIP(filtered_backlog, schedule, start_date_str, end_date)
    
    # Append the result to the list of DataFrames
    dfs.append(res)

# Concatenate all the DataFrames into one big DataFrame
results_df = pd.concat(dfs, ignore_index=True)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Separate scheduled and not scheduled tasks
scheduled_df = results_df[results_df.Date != "Not Scheduled"]
not_scheduled_df = results_df[results_df.Date == "Not Scheduled"]

# Process scheduled tasks
scheduled_df = scheduled_df.sort_values('Date').drop_duplicates('case_number_task', keep='last')
scheduled_df['Date'] = pd.to_datetime(scheduled_df['Date'])
daily_team_hours = scheduled_df.groupby(['teamcode', 'Date'])['Normtijd (in hours)'].sum().reset_index()

# Process not scheduled tasks
not_scheduled_df['Deadline'] = pd.to_datetime(not_scheduled_df['Deadline'])
not_scheduled_hours_per_day = not_scheduled_df.groupby(['teamcode', 'Deadline'])['Normtijd (in hours)'].sum().reset_index()

# Filter the schedule hours
start_date = daily_team_hours['Date'].min()
end_date = '2023-03-01'
schedule_hours = schedule[
    (schedule['Datum'] >= start_date) & 
    (schedule['Datum'] <= end_date) &
    (schedule['Datum'].dt.weekday < 5)  # Monday=0, Sunday=6
]
schedule_hours = schedule_hours[['Datum', "Team", "TotalHours"]].rename(columns={'Datum': 'Date', 'Team': 'teamcode'})

# Merge scheduled tasks with schedule hours
plot_data = pd.merge(daily_team_hours, schedule_hours, on=['teamcode', 'Date'], how='outer')
plot_data = plot_data.sort_values(by=["teamcode", "Date"])

# Create a plot for each team
teams = plot_data['teamcode'].unique()

for team in teams:
    team_schedule = schedule_hours[schedule_hours['teamcode'] == team].copy()  # Make a copy here    
    team_scheduled = daily_team_hours[daily_team_hours['teamcode'] == team]
    team_unscheduled = not_scheduled_hours_per_day[not_scheduled_hours_per_day['teamcode'] == team]
    
    # Ensure 'Date' in team_schedule is converted to datetime for accurate plotting
    team_schedule['Date'] = pd.to_datetime(team_schedule['Date'])

    # Create a date range that includes all days, filling the gaps (e.g., weekends)
    all_dates = pd.date_range(start=team_schedule['Date'].min(), end=team_schedule['Date'].max())

    # Merge scheduled and unscheduled dataframes with all_dates to ensure all dates are included
    all_dates_df = pd.DataFrame(all_dates, columns=['Date'])
    combined_hours = pd.merge(all_dates_df, team_scheduled, on='Date', how='left')
    combined_hours = pd.merge(combined_hours, team_unscheduled, left_on='Date', right_on='Deadline', how='left', suffixes=('_scheduled', '_unscheduled')).fillna(0)

    # Plot
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Scheduled task hours
    ax.bar(combined_hours['Date'], combined_hours['Normtijd (in hours)_scheduled'], width=0.4, label='Scheduled Hours', align='center', color='blue')

    # Unscheduled task hours
    ax.bar(combined_hours['Date'], combined_hours['Normtijd (in hours)_unscheduled'], width=0.4, label='Not Scheduled Tasks', align='center', color='orange', bottom=combined_hours['Normtijd (in hours)_scheduled'])

    # Available hours with gaps for weekends or non-working days
    available_hours_mask = team_schedule['TotalHours'] > 0
    ax.plot(team_schedule.loc[available_hours_mask, 'Date'], team_schedule.loc[available_hours_mask, 'TotalHours'], color='red', marker='o', linestyle='-', linewidth=2, markersize=8, label='Available Hours')

    # Formatting
    ax.set_xlabel('Date')
    ax.set_ylabel('Hours')
    ax.set_title(f'Task Schedule for Team: {team}')
    ax.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f'Images/HeuristicSchedule_{team}.png')
    plt.show()
